In [69]:
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import os
import numpy as np
import lightgbm as lgb
import xgboost as xgb

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import optuna 
from optuna.samplers import TPESampler 

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

from sklearn.model_selection import KFold, train_test_split,RepeatedKFold
from sklearn.metrics import mean_squared_error

from sklearn.metrics import r2_score

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [70]:
df = pd.read_csv('船舶装卸货量预测-训练集-20240611.csv',encoding='gb2312')

In [71]:
df.head(913)

,航次ID,船舶ID,进泊时间,离泊时间,船长,船宽,载重吨,船舶类型代码A,船舶类型代码B,泊位位置,装载量
0,1,416047000,1675473797,1675518110,191,28,21008,B,A,121.92547188203457 29.936120717921202,727.00
1,2,477348700,1705873575,1705955326,148,23,13295,B,A,122.03102823282664 29.89508922632821,4655.30
2,3,403581000,1698565729,1698648243,183,32,45963,C,B,121.9529721578593 29.94995541645756,5232.04
3,4,636017880,1682725395,1682902247,333,60,319611,D,C,121.9900784144446 29.92800063526856,132480.73
4,5,352002194,1701590148,1701743369,247,42,106808,D,C,121.8177632951752 29.95400256789684,94283.68
...,...,...,...,...,...,...,...,...,...,...,...
908,909,447097000,1696903694,1697179655,333,60,319660,D,C,121.8177632951752 29.95400256789684,272734.00
909,910,566013000,1673934926,1674083618,224,36,54505,A,C,121.97377818306916 29.945100713933336,24206.43
910,911,255805609,1701659835,1701759020,300,50,207562,E,D,122.07685838333425 29.893054494052702,93309.00
911,912,477311600,1703843059,1703921019,0,0,49971,B,A,122.03102823282664 29.89508922632821,1155.00


In [72]:
df.loc[491, '船长'] = df['船长'].mean().astype(int)
df.loc[491, '船宽'] = df['船宽'].mean().astype(int)
df.loc[202, '船长'] = df['船长'].mean().astype(int)
df.loc[202, '船宽'] = df['船宽'].mean().astype(int)
df.loc[911, '船长'] = df['船长'].mean().astype(int)
df.loc[911, '船宽'] = df['船宽'].mean().astype(int)

In [73]:
len(df[df['船长']==0])

0

In [74]:
def feature(df):
    df['离泊时间'] = df['离泊时间'].replace({' None': np.nan}).astype(float)
    df['time_diff'] = df['离泊时间'] - df['进泊时间']
#     mean_value = df['time_diff'].mean().astype(int)
#     df['time_diff'] = df['time_diff'].fillna(mean_value)
#     df['离泊时间'] = df['进泊时间'] + df['time_diff']
#     df['离泊时间'] = df['离泊时间'].astype(int)
    df['进泊时间'] = pd.to_datetime(df['进泊时间'], unit='s')
    df['离泊时间'] = pd.to_datetime(df['离泊时间'], unit='s')

    for f in ['进泊时间', '离泊时间']:
        df[f+'_month'] = df[f].dt.month
        #df[f+'_day'] = df[f].dt.day
        df[f+'_hour'] = df[f].dt.hour
        df[f+'_dayofweek'] = df[f].dt.dayofweek
        # df[f+'_birthday'] = df[f].dt.is_month_end  # 判断是否是一个月的最后一天
        # df[f+'_3end'] = df[f].dt.is_quarter_end  # 判断是否是季度的最后一天
        #df[f+'_jidu'] = df[f].dt.quarter  # 判断是一年的第几个季度

    df['A_le'] = df['船舶类型代码A'].factorize()[0]
    df['B_le'] = df['船舶类型代码B'].factorize()[0]
    
    df['AB_le'] = (df['船舶类型代码A'] + '_' + df['船舶类型代码B']).factorize()[0]
    
    df['面积'] = df['船长'] * df['船宽']
    #df['比例'] = df['船长'] / df['船宽']
    #df['系数'] = df['载重吨']/df['面积']
    #df['积载因数'] = df['面积']/df['载重吨']
    
    # for num_f in ['载重吨', 'time_diff',]:
    #     for cat_f in ['船舶类型代码B']:
    #         df[cat_f + '_' + num_f + '_mean'] = df.groupby(cat_f)[num_f].transform('mean')
    #         df[cat_f + '_' + num_f + '_std'] = df.groupby(cat_f)[num_f].transform('std')
    #         df[cat_f + '_' + num_f + '_max'] = df.groupby(cat_f)[num_f].transform('max')
    #         df[cat_f + '_' + num_f + '_min'] = df.groupby(cat_f)[num_f].transform('min')
    
    df['经度'] = df['泊位位置'].map(lambda x: float(x.split(' ')[0]))
    df['纬度'] = df['泊位位置'].map(lambda x: float(x.split(' ')[1]))

    s = (df.dtypes == 'object')
    object_cols = ['船舶类型代码A','船舶类型代码B']
    OH_X_train = pd.get_dummies(df[object_cols], prefix=None,prefix_sep='',dtype=int)
    OH_X_train = df.join(OH_X_train)
    df = OH_X_train
    
    le = LabelEncoder()
    label = le.fit_transform(df['船舶类型代码A'])
    df.drop('船舶类型代码A',axis=1, inplace=True)
    df["船舶类型代码A"] = label
    
    label1 = le.fit_transform(df['船舶类型代码B'])
    df.drop('船舶类型代码B',axis=1, inplace=True)
    df["船舶类型代码B"] = label1

    df = df.drop(['航次ID','船舶ID'],axis=1)
    
    return df

In [75]:
df= feature(df)

In [79]:
df.head(861)

,进泊时间,离泊时间,船长,船宽,载重吨,泊位位置,装载量,time_diff,进泊时间_month,进泊时间_hour,...,船舶类型代码AC,船舶类型代码AD,船舶类型代码AE,船舶类型代码AF,船舶类型代码BA,船舶类型代码BB,船舶类型代码BC,船舶类型代码BD,船舶类型代码A,船舶类型代码B
0,2023-02-04 01:23:17,2023-02-04 13:41:50,191,28,21008,121.92547188203457 29.936120717921202,727.00,44313.0,2,1,...,0,0,0,0,1,0,0,0,1,0
1,2024-01-21 21:46:15,2024-01-22 20:28:46,148,23,13295,122.03102823282664 29.89508922632821,4655.30,81751.0,1,21,...,0,0,0,0,1,0,0,0,1,0
2,2023-10-29 07:48:49,2023-10-30 06:44:03,183,32,45963,121.9529721578593 29.94995541645756,5232.04,82514.0,10,7,...,1,0,0,0,0,1,0,0,2,1
3,2023-04-28 23:43:15,2023-05-01 00:50:47,333,60,319611,121.9900784144446 29.92800063526856,132480.73,176852.0,4,23,...,0,1,0,0,0,0,1,0,3,2
4,2023-12-03 07:55:48,2023-12-05 02:29:29,247,42,106808,121.8177632951752 29.95400256789684,94283.68,153221.0,12,7,...,0,1,0,0,0,0,1,0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,2023-06-11 08:44:25,2023-06-12 08:15:39,292,45,182511,122.07685838333425 29.893054494052702,78550.00,84674.0,6,8,...,0,0,1,0,0,0,0,1,4,3
857,2024-01-22 19:24:02,2024-01-24 01:19:15,172,28,24518,121.92547188203457 29.936120717921202,1989.00,107713.0,1,19,...,0,0,0,0,1,0,0,0,1,0
858,2023-08-10 09:46:07,2023-08-11 11:51:08,353,54,178257,122.04040625503215 29.89264034445189,7000.00,93901.0,8,9,...,0,0,0,0,1,0,0,0,1,0
859,2023-05-31 05:21:55,2023-05-31 05:25:46,230,32,50513,121.78323817164318 29.97243550629892,22728.99,231.0,5,5,...,0,0,0,0,0,0,1,0,0,2


In [80]:
test_feature = pd.read_csv('船舶装卸货量预测-测试集X-20240611.csv',encoding='gb2312')

In [81]:
test= feature(test_feature)

In [82]:
test

,进泊时间,离泊时间,船长,船宽,载重吨,泊位位置,time_diff,进泊时间_month,进泊时间_hour,进泊时间_dayofweek,...,船舶类型代码AC,船舶类型代码AD,船舶类型代码AE,船舶类型代码AF,船舶类型代码BA,船舶类型代码BB,船舶类型代码BC,船舶类型代码BD,船舶类型代码A,船舶类型代码B
0,2023-05-02 01:04:58,2023-05-03 00:38:19,97,16,3844,121.8960833011353 29.93290032975045,84801.0,5,1,1,...,0,0,0,0,0,1,0,0,0,1
1,2023-03-03 09:52:55,2023-03-03 19:39:33,147,24,13075,121.92547188203457 29.936120717921202,35198.0,3,9,4,...,0,0,0,0,1,0,0,0,1,0
2,2023-01-28 13:09:53,2023-01-29 02:17:51,366,48,140637,122.02107120092448 29.778912029292396,47278.0,1,13,5,...,0,0,0,0,1,0,0,0,1,0
3,2023-12-28 06:03:31,2023-12-31 04:27:04,156,25,26004,121.75439821138373 29.979150371202415,253413.0,12,6,3,...,1,0,0,0,0,1,0,0,2,1
4,2023-08-15 02:07:43,2023-08-16 03:07:28,119,19,5562,121.8960833011353 29.93290032975045,89985.0,8,2,1,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,2023-01-19 06:38:30,2023-01-21 04:54:14,399,56,174239,122.0464197431265 29.8909733510863,166544.0,1,6,3,...,0,0,0,0,1,0,0,0,1,0
899,2023-07-15 08:16:23,2023-07-17 02:19:31,226,36,54747,121.97377818306916 29.945100713933336,151388.0,7,8,5,...,0,0,0,0,0,0,1,0,0,2
900,2023-03-08 00:03:43,2023-03-09 06:22:47,229,38,93217,122.07685838333425 29.893054494052702,109144.0,3,0,2,...,0,0,1,0,0,0,0,1,4,3
901,2023-07-16 07:07:32,2023-07-16 18:34:28,148,23,12631,121.93228463787132 29.940084640162464,41216.0,7,7,6,...,0,0,0,0,1,0,0,0,1,0


In [83]:
feats1 = [f for f in df.columns if f not in ['装载量','泊位位置','进泊时间','离泊时间']]
train_X = df[feats1]
train_y = df['装载量']
feats2 = [f for f in test.columns if f not in ['泊位位置','进泊时间','离泊时间']]
test = test[feats2]

In [84]:
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.4,random_state=42) 

In [85]:
params_lgb = {
    'num_iterations': 2000,
    'loss_function': 'RMSE',
    'verbose': -1,
    'boosting_type': 'gbdt',
    'objective': 'rmse',
    'num_leaves': 255,
    'learning_rate': 0.01,
    'min_data_in_leaf': 20,
    'one_hot_max_size': 2,
    'max_depth': 8,
    "max_bin":48,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'n_estimators':800,
}
model_lgb = []

In [86]:
params_xgb = {
    "num_boost_round": 500,
    "learning_rate": 0.05,
    "booster": "dart",
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "max_leaves": 255,
    "verbosity": 1,
    #"seed": 42,
    "nthread": -1,
    "colsample_bytree": 0.6,
    "subsample": 0.7,
    "early_stopping_rounds": 200,
    'max_depth':8,
#     'min_child_weight':1,
#     'reg_alpha':0.1,
#     'reg_lambda':0.1,
}
model_xgb = []

In [109]:
kfold = KFold(n_splits=4, random_state=42, shuffle=True)

x = train_X
y = train_y
mse = 0
for fold, (train_index, val_index) in enumerate(kfold.split(x, y)):
    logging.info(f'############ fold: {fold} ###########')
    x_train, x_val, y_train, y_val = x.iloc[train_index], x.iloc[val_index], y.iloc[train_index], y.iloc[val_index]
    
    trainset = lgb.Dataset(x_train, y_train)
    valset = lgb.Dataset(x_val, y_val)
    model = lgb.train(params_lgb, trainset, valid_sets=[trainset, valset], callbacks=[lgb.log_evaluation(1000)])
    model_lgb.append(model)
    lgb_pred = pd.Series(model.predict(x_val, num_iteration=model.best_iteration), index=y_val.index).fillna(0)
    
    trainset = xgb.DMatrix(x_train, y_train, enable_categorical=True, nthread=-1)
    valset = xgb.DMatrix(x_val, y_val, enable_categorical=True, nthread=-1)
    model = xgb.train(params_xgb, trainset, evals=[(trainset, 'train'),(valset, 'eval')], num_boost_round=params_xgb["num_boost_round"], early_stopping_rounds=params_xgb["early_stopping_rounds"], verbose_eval=1000)
    model_xgb.append(model)
    xgb_pred = pd.Series(model.predict(valset, iteration_range=(0, model.best_ntree_limit)), index=y_val.index).fillna(0)
    
    val_pred = (lgb_pred + xgb_pred) / 2
    mse += mean_squared_error(y_val.fillna(0), val_pred)
rmse = np.sqrt(mse / kfold.n_splits)
score = rmse
logging.info(f"--------------本地分数 {score}--------------")

2024-07-06 01:21:06,029 : INFO : ############ fold: 0 ###########


[01:21:10] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stopping_rounds", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:62434.53229	eval-rmse:49798.82260
[268]	train-rmse:3192.45052	eval-rmse:17123.01409


2024-07-06 01:21:30,412 : INFO : ############ fold: 1 ###########


[01:21:34] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stopping_rounds", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:60077.64703	eval-rmse:58170.56535
[290]	train-rmse:2937.61734	eval-rmse:20861.10183


2024-07-06 01:21:57,791 : INFO : ############ fold: 2 ###########


[01:22:01] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stopping_rounds", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:57528.52339	eval-rmse:65288.87545
[290]	train-rmse:1787.63997	eval-rmse:22716.36616


2024-07-06 01:22:25,310 : INFO : ############ fold: 3 ###########


[01:22:29] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stopping_rounds", "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:58028.76056	eval-rmse:63887.35733
[282]	train-rmse:1707.23766	eval-rmse:20552.32907


2024-07-06 01:22:51,927 : INFO : --------------本地分数 20106.98367349869--------------


In [110]:
v_pred = np.zeros((y_valid.shape[0], ))
for i in range(0, kfold.n_splits):
    v_pred += model_lgb[i].predict(X_valid, num_iteration=model_lgb[i].best_iteration)
    v_pred += model_xgb[i].predict(xgb.DMatrix(X_valid, enable_categorical=True, nthread=-1), iteration_range=(0, model_xgb[i].best_ntree_limit))
v_pred = v_pred / 2 / kfold.n_splits

In [111]:
rmse = mean_squared_error(y_valid, v_pred, squared=False)
rmse

9885.504772870752

In [112]:
x_test_b = test
y_pred = np.zeros((test.shape[0], ))
for i in range(0, kfold.n_splits):
    y_pred += model_lgb[i].predict(x_test_b, num_iteration=model_lgb[i].best_iteration)
    y_pred += model_xgb[i].predict(xgb.DMatrix(x_test_b, enable_categorical=True, nthread=-1), iteration_range=(0, model_xgb[i].best_ntree_limit))
y_pred = y_pred / 2 / kfold.n_splits

In [113]:
y_pred = np.around(y_pred, decimals=1)

In [114]:
y_pred = pd.DataFrame(y_pred)
y_pred

,0
0,2364.3
1,1831.0
2,5353.9
3,13599.2
4,3680.3
...,...
898,14890.1
899,18780.4
900,29922.6
901,1726.3


In [115]:
y_pred.to_csv('result.txt',sep='\t',index=False,header=None)